#### We will start by making dataset suitable for further processing.

In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np

#importing the dataset 
ah_data=pd.read_csv('C:/Users/admin/Desktop/EeD/Python/Project/AmesHousingData.csv')
        
#dropping Id column
ah_data = ah_data.drop(['Id'], axis = 1)

#changing the default column and row display settings to display all cloumns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
ah_data.sample(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1246,60,FV,65.0,8125,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,7,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,756,756,GasA,Ex,Y,SBrkr,756,797,0,1553,0,0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2005.0,RFn,2,615,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,3,2006,New,Partial,186500
420,90,RM,78.0,7060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,Duplex,SFoyer,7,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,200.0,TA,Gd,PConc,Gd,Gd,Gd,GLQ,1309,Unf,0,35,1344,GasA,Ex,Y,SBrkr,1344,0,0,1344,2,0,2,0,2,2,TA,8,Typ,0,NaN,Attchd,1997.0,Fin,4,784,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,11,2008,WD,Alloca,206300
672,20,RL,NaN,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Veenker,Norm,Norm,1Fam,1Story,6,6,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,Gd,TA,CBlock,Gd,TA,No,ALQ,767,Unf,0,441,1208,GasA,TA,Y,SBrkr,1208,0,0,1208,1,0,1,1,3,1,TA,6,Typ,1,TA,Attchd,1977.0,RFn,2,546,TA,TA,Y,198,42,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,165000
20,60,RL,101.0,14215,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,380.0,Gd,TA,PConc,Ex,TA,Av,Unf,0,Unf,0,1158,1158,GasA,Ex,Y,SBrkr,1158,1218,0,2376,0,0,3,1,4,1,Gd,9,Typ,1,Gd,BuiltIn,2005.0,RFn,3,853,TA,TA,Y,240,154,0,0,0,0,NaN,NaN,NaN,0,11,2006,New,Partial,325300
406,50,RL,51.0,10480,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,1.5Fin,6,5,1936,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,1064,1064,GasA,Ex,Y,FuseA,1166,0,473,1639,0,0,1,0,3,1,TA,6,Maj2,0,NaN,Detchd,1936.0,Unf,1,240,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2008,WD,Normal,115000


In [2]:
#getting to know the dimensions of the dataset
print(ah_data.shape)

(1460, 80)


In [3]:
#splitting the variables in dataset into predictors and target variable
X = ah_data.drop(['SalePrice'],axis = 1)
y = ah_data.SalePrice

In [4]:
#splitting the dataset into train and test sets
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y = train_test_split(X,y,test_size = 0.2,random_state = 9)

#printing the dimensions of train and test set of predictors and target variable
for item in [train_X,val_X,train_y,val_y]:
    print(item.shape)

(1168, 79)
(292, 79)
(1168,)
(292,)


In [5]:
'''Creating a custom function for imputation with different strategies like 
imputing mean or median or mode''' 

from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean',filler='NA'):
       self.strategy = strategy
       self.fill = filler
    
    def fit(self, X, y=None):
       if self.strategy in ['mean','median']:
           if not all(X.dtypes == np.number):
               raise ValueError('dtypes mismatch np.number dtype is \
                                 required for '+ self.strategy)
       if self.strategy == 'mean':
           self.fill = X.mean()
       elif self.strategy == 'median':
           self.fill = X.median()
       elif self.strategy == 'mode':
           self.fill = X.mode().iloc[0]
       elif self.strategy == 'fill':
           if type(self.fill) is list and type(X) is pd.DataFrame:
               self.fill = dict([(cname, v) for cname,v in zip(X.columns, self.fill)])
       return self

    def transform(self, X, y=None):
       return X.fillna(self.fill)

In [6]:
#using the custom function created to impute mode of the columns  
train_X = CustomImputer(strategy = 'mode').fit_transform(train_X)
val_X = CustomImputer(strategy = 'mode').fit_transform(val_X)
train_y = CustomImputer(strategy = 'mode').fit_transform(train_y)
val_y = CustomImputer(strategy = 'mode').fit_transform(val_y)

In [7]:
#seggregating columns as per data type 
integer_columns = train_X.select_dtypes(include=['int64']).columns.tolist()
#integer_columns = integer_columns[1:65]
char_text_columns = train_X.select_dtypes(include=['object']).columns.tolist()
float_columns = train_X.select_dtypes(include = ['float64']).columns.tolist()

#we are picking out only integer and float columns and clubbing them together as numbers column
numbers_columns = float_columns + integer_columns

#confirming that columns add up and nothing is missed
all_columns = float_columns + integer_columns + char_text_columns
print(len(all_columns))

79


In [8]:
train_X = train_X[all_columns]
val_X=val_X[all_columns]